# HW 2 - Разложение матриц градиентным методом

Цель задания: В ходе реализации [разложения Таккера](https://proceedings.neurips.cc/paper/2018/file/45a766fa266ea2ebeb6680fa139d2a3d-Paper.pdf) градиентным методом освоить pyTorch и реализовать подходы оптимизации параметров модели (в отсутствии готовых решений).

[Более-менее внятное описание алгоритма канонического разложения](https://www.alexejgossmann.com/tensor_decomposition_tucker/) - само аналитическое разложение вам реализовывать НЕ НУЖНО

In [12]:
import random
import time
import torch
import pandas as pd
import numpy as np

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from numpy.linalg import svd, matrix_rank, pinv, inv
from scipy.linalg import eigh, eig
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
from torch import nn

import tensorly as tl
tl.set_backend('pytorch')
from tensorly.decomposition import tucker
from tensorly.tucker_tensor import tucker_to_tensor

torch.manual_seed(0)

## 1 Создайте 3х мерный тензор
Размер тензора не меньше 100 по каждой из размерностей.

Заполните случайными целыми числами в диапазоне от 0 до 9.

Примечание: разложение будет корректно работать со случайным тензором, только если изначально создавать случайные ядро и матрицы, а потом по ним формировать тензор. Работайте с типом *torch.Tensor.double*.

In [13]:
# Создадим тензор: размер тензора и r задаётся
def get_tensor(size=(100,200,150), r=(10,10,10)):
    # data - тензор с заданной размерностью
    # U - список матриц
    # G - ядро разложения
    
    assert np.all(np.array(size)>=100), 'Размер тензора не меньше 100 по каждой из размерностей.'
    
    G = torch.randint(10, (r[0], r[1], r[2]), requires_grad=True, dtype=torch.double)
    U = [torch.randint(10, (size[0], r[0]), requires_grad=True, dtype=torch.double), 
         torch.randint(10, (size[1], r[1]), requires_grad=True, dtype=torch.double), 
         torch.randint(10, (size[2], r[2]), requires_grad=True, dtype=torch.double)]
    
    data = G
    
    #print(U[0].shape, data.shape)
    data = torch.tensordot(U[0], data, dims=([1], [0]))
    
    #print(U[1].shape, data.shape)
    data = torch.tensordot(U[1], data.permute([1, 0, 2]), dims=([1], [0]))
    data = data.permute([1, 0, 2])
    
    #print(U[2].shape, data.shape)
    data = torch.tensordot(U[2], data.permute([2, 1, 0]), dims=([1], [0]))
    data = data.permute([2, 1, 0])
    
    return data, U, G

Сгенерируйте тензор и добавьте к нему случайный шум с размерностью *1e-2*

In [14]:
data, U, G = get_tensor(r = (10, 20, 30))
data.shape, [u.shape for u in U], G.shape

(torch.Size([100, 200, 150]),
 [torch.Size([100, 10]), torch.Size([200, 20]), torch.Size([150, 30])],
 torch.Size([10, 20, 30]))

In [15]:
noise = torch.randint(10, (data.shape))*1e-2
data += noise

Вопрос:
Почему задание не имеет смысла для полностью случайного тензора и зачем добавлять шум? *не отвечать нельзя*

Ответ: Если тензор полностью случайный, то основная цель факторизации - выявление закономерностей и зависимостей - теряется, что затрудняет работу градиентного спуска. Кроме того, шум помогает градиентному спуску справляться с проблемой локальных минимумов.


## 2 Реализуйте метод для восстановления тензора по разложению

In [16]:
# Функция, восстанавливающая тензор по ядру и матрицам
def repair_tensor(G_, U):
    # data - восстановленный тензор из матриц и ядра
    # U - список матриц
    # G_ - ядро разложения
    
    data = G_
    
    for i in range(len(U)):
        numbers = list(range(G_.dim()))
        numbers[0], numbers[i] = numbers[i], numbers[0]
        data = torch.tensordot(U[i], data.permute(numbers), dims=([1], [0]))
        data = data.permute(numbers)
    
    return data

In [17]:
r1 = (15, 30, 45)
_, U1, G1 = get_tensor(size=(150, 200, 250), r=r1)

data1 = repair_tensor(G1, U1)
print('data1:', data1.shape)

data1_p = tucker_to_tensor((G1, U1))
print('data1_p:', data1_p.shape)

print('\n', mean_squared_error(data1.detach().numpy().flatten(), data1_p.detach().numpy().flatten()))
print((data1==data1_p).all())

data1: torch.Size([150, 200, 250])
data1_p: torch.Size([150, 200, 250])

 0.0
tensor(True)


## 3 Сделайте разложение библиотечным методом
Пакет можете брать любой

In [18]:
G2, U2 = tucker(data1, r1)

Не забудьте померить ошибку разложения по метрике MSE

In [19]:
re_data1 = tucker_to_tensor((G2, U2))
mean_squared_error(data1.detach().numpy().flatten(), re_data1.detach().numpy().flatten())

2.9151234218783665e-16

## 4 Реализуйте разложение градиентным методом

### 4.1 Реализуйте *optimizer*
Можно взять из исходников *PyTorch* и отнаследоваться от *torch.optim.optimizer*.
Используйте квадратичный *Loss*.

In [20]:
import math
import torch
from torch.optim.optimizer import Optimizer

class Opt(Optimizer):

    def __init__(self, params, lr=1e-3):

        defaults = dict(
            lr=lr
        )
        super().__init__(params, defaults)

    
    def step(self, epoch, closure=None):
        """Perform a single optimization step.

        Args:
            closure (Callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()
        for group in self.param_groups:
            lr = group['lr']
            
            for p in group['params']:
                if p.grad is None:
                    continue
                
                if not hasattr(p, 'state'):
                    p.state = {'m': torch.zeros_like(p), 'v': torch.zeros_like(p), 't': 0}
                
                state = p.state
                state['t'] += 1

                m = state['m']
                v = state['v']

                m = 0.9*m + p.grad.data/10
                v = 0.99*v + torch.square(p.grad.data)/100

                m_hat = m / (1 - 0.9 ** state['t'])
                v_hat = v / (1 - 0.99 ** state['t'])

                p.data = p.data - lr * m_hat / (torch.sqrt(v_hat) + 1e-8)
                
        return loss

### 4.2 Реализуйте цикл оптимизации параметров

Стоит параметры оптимизировать сразу на GPU

In [21]:
%%time
r = (5, 10, 5)
size = (100, 150, 110)

data_train, _u, _g = get_tensor(size=size, r=r)
noise = torch.randint(10, (data_train.shape))*1e-2
data_train += noise
data_train = data_train.detach()

G = torch.randint(10, (r[0], r[1], r[2]), requires_grad=True, dtype=torch.double)
U = [torch.randint(10, (size[0], r[0]), requires_grad=True, dtype=torch.double), 
     torch.randint(10, (size[1], r[1]), requires_grad=True, dtype=torch.double), 
     torch.randint(10, (size[2], r[2]), requires_grad=True, dtype=torch.double)]

model = [nn.Parameter(U[0]), nn.Parameter(U[1]), nn.Parameter(U[2]), nn.Parameter(G)]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_train = data_train.to(device)
for param in model:
    param.to(device)

lr = 1e-2
criterion = nn.MSELoss()
optimizer = Opt(model, lr=lr)


# print(f"PARAM 0 before: {model[0]}")
# print(f"PARAM 1 before: {model[1]}")
# print(f"PARAM 2 before: {model[2]}")
# print(f"PARAM 3 before: {model[3]}\n")

for epoch in range(1000):
    optimizer.zero_grad()
    
    data_predicted = repair_tensor(model[3], [model[0], model[1], model[2]])
    loss = criterion(data_predicted, data_train)  
    
    loss.backward()

    optimizer.step(epoch)  

print(loss)


tensor(9877241.5033, dtype=torch.float64, grad_fn=<MseLossBackward0>)
CPU times: total: 3min 12s
Wall time: 34.7 s


## 5 Приведите сравнение скорости работы и ошибки восстановления методом из пакета и реализованного градиентного
Сравнение может считаться ± объективным с размером выборки от 10.

In [23]:
predicted_data_grad = repair_tensor(model[3], [model[0], model[1], model[2]])

mse_grad = mean_squared_error(data_train.detach().numpy().flatten(), predicted_data_grad.detach().numpy().flatten())

print(mse_grad)

9921703.266557436


In [24]:
%%time
mses = np.array([])

for i in range(10):
    data, U, G = get_tensor()
    noise = torch.randint(10, (data.shape))*1e-2
    data += noise
    
    re_data = tucker_to_tensor((G, U))
    mses = np.append(mses, mean_squared_error(data.detach().numpy().flatten(), re_data.detach().numpy().flatten()))

print('mse_package =', np.mean(mses))

mse_package = 0.0028503482531408
CPU times: total: 12.2 s
Wall time: 2.19 s
